# Delivered Cost Modeling Approach
1. Draw a polygon on the map that spatially identifies the extent of the study area.
2. Draw a point on the map that spatially locates conversion facility(s) for biomass delivery.
3. Specify various machine rates.
4. Click the run button. Delivered cost surfaces will be added to the Map. 

In [ ]:
import delvCost
import ipyleaflet
import ipywidgets
import localtileserver
import ipyfilechooser
from IPython.display import HTML
import base64

ly = ipywidgets.Layout(height='800px')

lc = ipyleaflet.LayersControl()
lc.position='topright'
dr=ipyleaflet.DrawControl()
dr.position='topleft'


bmaps=ipyleaflet.basemaps

m=ipyleaflet.Map(controls=[lc,dr],layout=ly,scroll_wheel_zoom=True,center=(41,-119),zoom=5)

tly1=ipyleaflet.basemap_to_tiles(bmaps.Esri.WorldImagery)
tly1.base = False

tly2=ipyleaflet.ImageService(
    url=r'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF230/US_230EVT/ImageServer',
    format='jpgpng',
    attribution='USDA USFS',
    name = 'Landfire EVT',
)


tly3=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Nat_BurnProbability/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name = 'Burn Probability'
)

tly4=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Potential_Control_Location/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',name='Potential Control Locatoins',
)
tly5=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/SnagHazard_eDart/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',name='Snag Hazard eDart',
)
tly6=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Suppression_Difficulty_Index_80/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',name='Suppression Difficulty Index 80',

)

m.add(tly1)
m.add(tly2)
m.add(tly3)
m.add(tly4)
m.add(tly5)
m.add(tly6)


#widgets
style = {'description_width': 'initial'}

tr_s=ipywidgets.widgets.FloatSlider(value=1.5,min=1.0,max=5.0,step=0.1,description='RT Skidder Speed (MPH):',style=style)
cb_s=ipywidgets.widgets.FloatSlider(value=2.0,min=1.0,max=5.0,step=0.1,description='Skyline Speed (MPH):',style=style)

tr_d=ipywidgets.widgets.FloatSlider(value=165,min=150,max=200,step=1,description='RT Skidder ($/hour):',style=style)
cb_d=ipywidgets.widgets.FloatSlider(value=400,min=300,max=500,step=1,description='Skyline ($/hour):',style=style)
fb_d=ipywidgets.widgets.FloatSlider(value=15,min=10,max=25,step=1,description='Fellerbuncher ($/CCF):',style=style)
hf_d=ipywidgets.widgets.FloatSlider(value=27,min=10,max=40,step=1,description='Hand Felling ($/CCF):',style=style)
pr_d=ipywidgets.widgets.FloatSlider(value=56,min=30,max=70,step=1,description='Processing ($/CCF):',style=style)
ha_d=ipywidgets.widgets.FloatSlider(value=98,min=70,max=120,step=1,description='Hauling ($/hour):',style=style)
ht_d=ipywidgets.widgets.FloatSlider(value=2470,min=0,max=4000,step=5,description='Hand Treatment ($/acre):',style=style)
pf_d=ipywidgets.widgets.FloatSlider(value=2470,min=0,max=4000,step=5,description='Prescribed Fire ($/acre):',style=style)

tr_p=ipywidgets.widgets.FloatSlider(value=1.25,min=1.0,max=5.0,step=0.1,description='RT Skidder Payload (CCF):',style=style)
cb_p=ipywidgets.widgets.FloatSlider(value=1.04,min=0.5,max=5.0,step=0.1,description='Skyline Payload (CCF):',style=style)
lt_p=ipywidgets.widgets.FloatSlider(value=12.25,min=5.0,max=20.5,step=0.1,description='Log Truck Payload (CCF):',style=style)

cb_o=ipywidgets.widgets.Checkbox(value=False,description='Create Optional surfaces', disabled=False, indent=False)

#optional widgets
roads_name=ipyfilechooser.FileChooser(filter_pattern=['*.shp'],title='Optional Roads')
barriers_name=ipyfilechooser.FileChooser(filter_pattern=['*.shp'],title='Optional Barriers')

prg_wdg=ipywidgets.widgets.IntProgress(value=0,min=0,max=12,description='Progress',style={'bar_color':'green'},orientation='horizontal')
out_wdg=ipywidgets.widgets.Output(layout={'border': '1px solid black'})

def get_local_tile_service(path, name='',palette='Greys',nodata=-999999,vmin=None, vmax=None):
    client1 = localtileserver.TileClient(path)
    t1 = localtileserver.get_leaflet_tile_layer(client1, band=[1], name=name, palette=palette,nodata=nodata,vmin=vmin,vmax=vmax)
    m.add(t1)
    return t1

def create_download_link(filename, title = "Click here to download: "):  
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title+f' {filename}',filename=filename)
    return HTML(html)

@out_wdg.capture()
def delv_cost(b):
    prg_wdg.value=0
    out_wdg.clear_output()
    outlyrlst=[]
    delvCost.sk_r=tr_s.value
    delvCost.cb_r=cb_s.value
    delvCost.sk_d=tr_d.value
    delvCost.cb_d=cb_d.value
    delvCost.fb_d=fb_d.value
    delvCost.hf_d=hf_d.value
    delvCost.pr_d=pr_d.value
    delvCost.lt_d=ha_d.value
    delvCost.pf_d=pf_d.value
    delvCost.sk_p=tr_p.value
    delvCost.cb_p=cb_p.value
    delvCost.lt_p=lt_p.value
    delvCost.cb_o=cb_o.value
    delvCost.pbar=prg_wdg

    #optional
    delvCost.lyr_roads_path=roads_name.value
    delvCost.lyr_barriers_path=barriers_name.value

    #check values
    pntlst=[]
    polylst=[]
    for d in dr.data:
        geo=d['geometry']
        t=geo['type']
        if(t=='Point'):pntlst.append(geo['coordinates'])
        if(t=='Polygon'):polylst.append(geo['coordinates'][0])
    
    if((len(pntlst)<1) | (len(polylst)<1)):
        print("You Must Create a Study Area and identify a Conversion Facility before clicking the Run button")
        return
    else:
        delvCost.study_area_coords = polylst
        delvCost.saw_coords = pntlst
        outdic=delvCost.run()
        for k in outdic.keys():
            vl = outdic[k]
            outlyrlst.append(get_local_tile_service(path=vl,name=k,palette='RdYlGn_r'))
            html=create_download_link(vl)
            display(html)
        
        return outlyrlst

  
btn=ipywidgets.widgets.Button(description='Run')
btn.on_click(delv_cost)


wb=ipywidgets.VBox([tr_s,cb_s,tr_d,cb_d,fb_d,hf_d,pr_d,ha_d,ht_d,pf_d,tr_p,
                   cb_p,lt_p,cb_o,roads_name,barriers_name,btn])

mb=ipywidgets.HBox([wb,m])

ipywidgets.VBox([mb,prg_wdg,out_wdg])